In [1]:
!pip install -qq -U mlxtend

     |████████████████████████████████| 1.3 MB 7.5 MB/s 


In [2]:
import pandas as pd
import numpy
from numpy.random import randint
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
from mlxtend.frequent_patterns import association_rules
import nltk
from nltk.tokenize import word_tokenize
import re
from nltk.stem import PorterStemmer
from tqdm.auto import tqdm
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from networkx.algorithms.flow import shortest_augmenting_path
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
PORTER_STEMMER = PorterStemmer()

In [4]:
def clean_tweets(x, STOPWORDS):
    # Lowercase
    sentence = x.lower()
    
    # Remove all non-alphabets (punctuation, numbers, new-line characters and extra-spaces)
    sentence = re.sub('http[s]?://\S+', '', sentence)
    sentence = re.sub(r'([^a-zA-Z ]+?)', '', sentence)
    #print(sentence)
    #sentence = sentence.replace('\n', '')
    # Remove URLs
    sentence = sentence.replace("world health organization", "who")
    #print(sentence)
    # Remove double spacing
    #sentence = re.sub('\s+', ' ', sentence)
    tokenized_tweet = [word for word in word_tokenize(sentence) if word not in STOPWORDS]
    tokenized_tweet = [PORTER_STEMMER.stem(word) for word in tokenized_tweet]
    return tokenized_tweet

<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \S
<ipython-input-4-e9114ee358bb>:6: DeprecationWarning: invalid escape sequence \S
  sentence = re.sub('http[s]?://\S+', '', sentence)


In [5]:
pharma_username = ['pfizer','JNJNews', 'AstraZeneca', 'GSK', 'Novartis']
# 
public_username = ['CDCgov', 'IHSgov', 'GovCanHealth', 'GCIndigenous']
ngo_username = ['WHO']

In [ ]:
stopwords_df = set(nltk.corpus.stopwords.words("english"));
for username in pharma_username:

  df = pd.read_csv('/content/drive/MyDrive/Project 2/data/pharma_companies/'+username+'.csv')
  
  #clean tweets
  tqdm.pandas()
  df["tokenized_tweet"] = df["tweet"].progress_apply(lambda x : clean_tweets(x, stopwords_df))
  tokenized_tweets = df["tokenized_tweet"].values.tolist()
  te = TransactionEncoder()
  te_ary = te.fit(tokenized_tweets).transform(tokenized_tweets)
  df = pd.DataFrame(te_ary, columns=te.columns_)

  min_sup=0.5
  frequent_itemsets = apriori(df, min_support=min_sup, use_colnames=True)
  print(len(frequent_itemsets.index))

  while(len(frequent_itemsets.index)==0):
    min_sup = min_sup/2
    frequent_itemsets = apriori(df, min_support=min_sup, use_colnames=True)
    print("fi:",len(frequent_itemsets.index))

  rules = association_rules(frequent_itemsets, ,metric="confidence")
  print("r:",len(rules.index))
  
  while(len(rules.index)<10):
    min_sup = min_sup/2
    frequent_itemsets = apriori(df, min_support=min_sup, use_colnames=True)
    print("fi:",len(frequent_itemsets.index))
    rules = association_rules(frequent_itemsets, metric="confidence")
    print("r:",len(rules.index))
  print(rules)

  df1 = (rules.explode('antecedents')
         .reset_index(drop=True)
         .explode('consequents')
         .reset_index(drop=True))
  print (df1)
  df1.to_csv('/content/drive/MyDrive/Project 2/rules/pharma_companies/'+username+'.csv');

  0%|          | 0/2474 [00:00<?, ?it/s]

0
fi: 0
fi: 7
r: 0
fi: 25
r: 0
fi: 118
r: 7
fi: 748
r: 2835
            antecedents                                   consequents  \
0                 (fda)                                      (approv)   
1              (licens)                                      (approv)   
2              (author)                                       (covid)   
3               (emerg)                                      (author)   
4                 (fda)                                      (author)   
...                 ...                                           ...   
2830     (use, prevent)        (licens, covid, approv, emerg, author)   
2831  (approv, prevent)           (licens, use, covid, emerg, author)   
2832   (emerg, prevent)          (licens, use, covid, approv, author)   
2833  (prevent, author)           (licens, use, covid, approv, emerg)   
2834           (licens)  (use, prevent, covid, approv, emerg, author)   

      antecedent support  consequent support   support  confide

  0%|          | 0/2368 [00:00<?, ?it/s]

0
fi: 5
r: 0
fi: 20
r: 0
fi: 239
r: 1889
     antecedents                                        consequents  \
0        (event)                                           (advers)   
1       (advers)                                            (event)   
2       (fdacdc)                                           (advers)   
3       (advers)                                           (fdacdc)   
4       (advers)                                           (report)   
...          ...                                                ...   
1884   (tollfre)    (advers, system, event, fdacdc, vaccin, report)   
1885    (advers)   (tollfre, system, event, fdacdc, vaccin, report)   
1886    (system)   (advers, tollfre, event, fdacdc, vaccin, report)   
1887     (event)  (advers, tollfre, system, fdacdc, vaccin, report)   
1888    (fdacdc)   (advers, tollfre, system, event, vaccin, report)   

      antecedent support  consequent support  support  confidence       lift  \
0               0.067568  

  0%|          | 0/1284 [00:00<?, ?it/s]

0
fi: 0
fi: 6
r: 0
fi: 36
r: 0
fi: 1159
r: 55982
      antecedents                                        consequents  \
0        (advers)                                          (discuss)   
1      (encourag)                                           (advers)   
2        (advers)                                         (encourag)   
3        (advers)                                            (event)   
4         (media)                                           (advers)   
...           ...                                                ...   
55977     (media)  (social, discuss, advers, encourag, event, top...   
55978     (topic)  (social, discuss, advers, encourag, event, med...   
55979    (websit)  (social, discuss, advers, encourag, event, med...   
55980    (report)  (social, discuss, advers, encourag, event, med...   
55981    (permit)  (social, discuss, advers, encourag, event, med...   

       antecedent support  consequent support   support  confidence  \
0              

  0%|          | 0/2359 [00:00<?, ?it/s]

0
fi: 0
fi: 4
r: 0
fi: 48
r: 27
                 antecedents        consequents  antecedent support  \
0                   (inform)          (contact)            0.097499   
1                    (media)         (investor)            0.095379   
2                 (investor)            (media)            0.100042   
3                    (local)            (offic)            0.093260   
4          (contact, inform)            (local)            0.080967   
5           (contact, local)           (inform)            0.071217   
6            (local, inform)          (contact)            0.069097   
7          (contact, inform)            (offic)            0.080967   
8           (contact, offic)           (inform)            0.070369   
9            (inform, offic)          (contact)            0.067401   
10          (contact, offic)            (local)            0.070369   
11          (contact, local)            (offic)            0.071217   
12            (local, offic)          (contac

  0%|          | 0/715 [00:00<?, ?it/s]

2
r: 0
fi: 12
r: 13
         antecedents       consequents  antecedent support  \
0            (local)         (contact)            0.267133   
1            (offic)         (contact)            0.278322   
2          (contact)           (pleas)            0.555245   
3            (pleas)         (contact)            0.528671   
4            (local)           (offic)            0.267133   
5            (offic)           (local)            0.278322   
6            (reach)           (thank)            0.261538   
7            (thank)           (reach)            0.314685   
8     (local, offic)         (contact)            0.265734   
9   (local, contact)           (offic)            0.258741   
10  (contact, offic)           (local)            0.265734   
11           (local)  (contact, offic)            0.267133   
12           (offic)  (local, contact)            0.278322   

    consequent support   support  confidence      lift  leverage  conviction  
0             0.555245  0.258741

In [ ]:
stopwords_df = set(nltk.corpus.stopwords.words("english"));
for username in public_username:

  df = pd.read_csv('/content/drive/MyDrive/Project 2/data/public_health_agencies/'+username+'.csv')
  
  #clean tweets
  tqdm.pandas()
  df["tokenized_tweet"] = df["tweet"].progress_apply(lambda x : clean_tweets(x, stopwords_df))
  tokenized_tweets = df["tokenized_tweet"].values.tolist()
  te = TransactionEncoder()
  te_ary = te.fit(tokenized_tweets).transform(tokenized_tweets)
  df = pd.DataFrame(te_ary, columns=te.columns_)

  min_sup=0.5
  frequent_itemsets = apriori(df, min_support=min_sup, use_colnames=True)
  print(len(frequent_itemsets.index))

  while(len(frequent_itemsets.index)==0):
    min_sup = min_sup/2
    frequent_itemsets = apriori(df, min_support=min_sup, use_colnames=True)
    print("fi:",len(frequent_itemsets.index))

  rules = association_rules(frequent_itemsets, metric="confidence")
  print("r:",len(rules.index))
  
  while(len(rules.index)<10):
    min_sup = min_sup/2
    frequent_itemsets = apriori(df, min_support=min_sup, use_colnames=True)
    print("fi:",len(frequent_itemsets.index))
    rules = association_rules(frequent_itemsets, metric="confidence")
    print("r:",len(rules.index))
  print(rules)

  df1 = (rules.explode('antecedents')
         .reset_index(drop=True)
         .explode('consequents')
         .reset_index(drop=True))
  print (df1)
  df1.to_csv('/content/drive/MyDrive/Project 2/rules/public_health_agencies/'+username+'.csv');

  0%|          | 0/7511 [00:00<?, ?it/s]

1
r: 0
fi: 2
r: 0
fi: 16
r: 1
fi: 40
r: 3
fi: 206
r: 33
         antecedents      consequents  antecedent support  consequent support  \
0          (booster)          (covid)            0.032219            0.559180   
1      (cdcdirector)             (rt)            0.037012            0.228465   
2       (cdcenviron)             (rt)            0.035415            0.228465   
3          (cdcmmwr)          (covid)            0.066036            0.559180   
4             (dose)          (covid)            0.035015            0.559180   
5             (mask)          (covid)            0.052856            0.559180   
6            (other)          (covid)            0.052190            0.559180   
7           (pandem)          (covid)            0.039009            0.559180   
8            (sever)          (covid)            0.041007            0.559180   
9             (slow)          (covid)            0.042205            0.559180   
10          (spread)          (covid)            0.11

  0%|          | 0/1632 [00:00<?, ?it/s]

0
fi: 5
r: 0
fi: 21
r: 1
fi: 73
r: 4
fi: 302
r: 99
               antecedents                consequents  antecedent support  \
0                 (alaska)                 (american)            0.067402   
1                 (alaska)                   (indian)            0.067402   
2                 (alaska)                    (nativ)            0.067402   
3                   (area)                       (ih)            0.042279   
4                 (cdcgov)                       (rt)            0.120711   
..                     ...                        ...                 ...   
94     (ih, indian, nativ)         (alaska, american)            0.039216   
95  (ih, indian, american)            (alaska, nativ)            0.034926   
96   (ih, nativ, american)           (alaska, indian)            0.035539   
97            (ih, alaska)  (indian, nativ, american)            0.036152   
98          (ih, american)    (alaska, nativ, indian)            0.039216   

    consequent support  

  0%|          | 0/52695 [00:00<?, ?it/s]

1
r: 0
fi: 10
r: 3
fi: 171
r: 1065
           antecedents                            consequents  \
0                (get)                                (covid)   
1              (learn)                                (covid)   
2                 (lt)                                (covid)   
3             (vaccin)                                (covid)   
4                (get)                               (vaccin)   
...                ...                                    ...   
1060  (vaccin, optout)        (learn, covid, lt, stop, repli)   
1061   (covid, optout)       (learn, repli, lt, stop, vaccin)   
1062      (lt, optout)    (learn, repli, covid, stop, vaccin)   
1063        (lt, stop)  (learn, repli, optout, covid, vaccin)   
1064       (lt, repli)   (learn, optout, covid, stop, vaccin)   

      antecedent support  consequent support   support  confidence      lift  \
0               0.205883            0.416548  0.171231    0.831690  1.996623   
1               0.231179

  0%|          | 0/3725 [00:00<?, ?it/s]

0
fi: 2
r: 0
fi: 6
r: 0
fi: 24
r: 0
fi: 107
r: 21
              antecedents  consequents  antecedent support  \
0              (canadian)         (rt)            0.041074   
1            (canadianpm)         (rt)            0.037584   
2                 (updat)      (covid)            0.066309   
3          (govcanhealth)         (rt)            0.040805   
4            (minindserv)         (rt)            0.057181   
5                (minist)         (rt)            0.060940   
6         (commun, updat)      (covid)            0.042685   
7        (nation, commun)      (first)            0.049933   
8         (first, commun)     (nation)            0.045638   
9         (nation, covid)      (first)            0.033557   
10        (covid, minist)       (live)            0.038121   
11         (minist, live)      (covid)            0.039195   
12          (covid, live)         (rt)            0.048322   
13        (covid, minist)         (rt)            0.038121   
14         (minist, 

In [ ]:
stopwords_df = set(nltk.corpus.stopwords.words("english"));
for username in ngo_username:

  df = pd.read_csv('/content/drive/MyDrive/Project 2/data/ngo_who/'+username+'.csv')
  
  #clean tweets
  tqdm.pandas()
  df["tokenized_tweet"] = df["tweet"].progress_apply(lambda x : clean_tweets(x, stopwords_df))
  tokenized_tweets = df["tokenized_tweet"].values.tolist()
  te = TransactionEncoder()
  te_ary = te.fit(tokenized_tweets).transform(tokenized_tweets)
  df = pd.DataFrame(te_ary, columns=te.columns_)

  min_sup=0.5
  frequent_itemsets = apriori(df, min_support=min_sup, use_colnames=True)
  print(len(frequent_itemsets.index))

  while(len(frequent_itemsets.index)==0):
    min_sup = min_sup/2
    frequent_itemsets = apriori(df, min_support=min_sup, use_colnames=True)
    print("fi:",len(frequent_itemsets.index))

  rules = association_rules(frequent_itemsets, metric="confidence")
  print("r:",len(rules.index))
  
  while(len(rules.index)<10):
    min_sup = min_sup/2
    frequent_itemsets = apriori(df, min_support=min_sup, use_colnames=True)
    print("fi:",len(frequent_itemsets.index))
    rules = association_rules(frequent_itemsets, metric="confidence")
    print("r:",len(rules.index))
  print(rules)

  df1 = (rules.explode('antecedents')
         .reset_index(drop=True)
         .explode('consequents')
         .reset_index(drop=True))
  print (df1)
  df1.to_csv('/content/drive/MyDrive/Project 2/rules/ngo_who/'+username+'.csv');

  0%|          | 0/24581 [00:00<?, ?it/s]

0
fi: 3
r: 0
fi: 8
r: 0
fi: 24
r: 0
fi: 84
r: 0
fi: 344
r: 481
            antecedents                                      consequents  \
0              (worker)                                         (health)   
1             (pahowho)                                        (whoafro)   
2             (pahowho)                                        (whoemro)   
3             (pahowho)                                       (whoeurop)   
4             (pahowho)                                       (whosearo)   
..                  ...                                              ...   
476  (whoemro, whowpro)           (pahowho, whoafro, whoeurop, whosearo)   
477  (whoafro, whoemro)           (pahowho, whoeurop, whosearo, whowpro)   
478  (whoafro, whowpro)           (whoemro, pahowho, whoeurop, whosearo)   
479           (pahowho)  (whoeurop, whosearo, whoemro, whowpro, whoafro)   
480          (whosearo)   (pahowho, whoeurop, whoemro, whowpro, whoafro)   

     antecedent support 